# Se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from catboost import CatBoostRegressor
%matplotlib inline

# Se lee el .csv

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test = pd.read_csv('csv/test.csv')

In [5]:
res_modelo2 = pd.read_csv('csv/solo_embedding.csv')
del res_modelo2['Unnamed: 0']
res_modelo2.head()

,0
0,0.771458
1,0.606724
2,0.879399
3,0.944014
4,0.894664


In [6]:
res_modelo2_test = pd.read_csv('csv/solo_embedding_test.csv')
del res_modelo2_test['Unnamed: 0']
res_modelo2_test.head()

,0
0,0.869460
1,0.561893
2,0.709307
3,0.601553
4,0.532795


In [7]:
train['res_modelo2'] = res_modelo2.iloc[:, 0]
train.head()

,id,keyword,location,text,target,res_modelo2
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.771458
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.606724
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.879399
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.944014
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.894664


In [8]:
test['res_modelo2'] = res_modelo2_test.iloc[:, 0]
test.head()

,id,keyword,location,text,res_modelo2
0,0,NaN,NaN,Just happened a terrible car crash,0.869460
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0.561893
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0.709307
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0.601553
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,0.532795


# Elaboración del modelo

Para este modelo se utilizarán árboles de decisión para realizar las predicciones y se tendrán en cuenta los siguientes features extraídos del análisis exploratorio, los cuales fueron seleccionados por el grupo como aquellos que podrían resultan más interesantes:

- Palabras con mayor y menor porcentaje de veracidad.
- Pares de 2 palabras con mayor y menor porcentaje de veracidad.
- Tweets que contienen @, ¿?, ¡! tienden a ser falsos.
- Keywords con mayor y menor porcentaje de veracidad.
- Locaciones más y menos veraces.
- Longitud.

### Preparación del set de datos

#### Parte inicial

In [9]:
#Palabras con mayor y menor porcentaje de veracidad
min_repetitions = ((0.2/100)*len(train.index))
words = train['text'].str.split()
clean_words = []

for sentence in words:
    clean_sentence = []
    for word in sentence:
        clean_word = re.sub('[^A-Za-z0-9]+','', word)
        if(clean_word != ''):
            clean_sentence.append(clean_word.lower())
    clean_words.append(clean_sentence)

In [10]:
train_aux = train
train_aux['words'] = clean_words
train_aux = train_aux.explode('words')
train_aux.head()

,id,keyword,location,text,target,res_modelo2,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.771458,our
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.771458,deeds
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.771458,are
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.771458,the
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.771458,reason


In [11]:
#Palabras con mayor y menor porcentaje de veracidad (TEST, CREO QUE SEPARA LOS TWEETS POR PALABRAS)***********
min_repetitions = ((0.2/100)*len(test.index))
words = test['text'].str.split()
clean_words = []

for sentence in words:
    clean_sentence = []
    for word in sentence:
        clean_word = re.sub('[^A-Za-z0-9]+','', word)
        if(clean_word != ''):
            clean_sentence.append(clean_word.lower())
    clean_words.append(clean_sentence)
test_aux = test
test_aux['words'] = clean_words
test_aux = test_aux.explode('words')
test_aux.head()

,id,keyword,location,text,res_modelo2,words
0,0,NaN,NaN,Just happened a terrible car crash,0.86946,just
0,0,NaN,NaN,Just happened a terrible car crash,0.86946,happened
0,0,NaN,NaN,Just happened a terrible car crash,0.86946,a
0,0,NaN,NaN,Just happened a terrible car crash,0.86946,terrible
0,0,NaN,NaN,Just happened a terrible car crash,0.86946,car


In [12]:
train_aux = train_aux.groupby('words').agg({'target':['sum','count']})
train_aux = train_aux[train_aux['target']['count']>min_repetitions]
train_aux['veracity'] = train_aux['target']['sum']/train_aux['target']['count']
train_aux.head()

target        veracity
         sum count          
words                       
0          9    12  0.750000
05        14    18  0.777778
06         7    10  0.700000
1         29    64  0.453125
10         9    23  0.391304

In [13]:
train_aux.reset_index(inplace=True) 

In [14]:
#Pares de 2 palabras con mayor y menor porcentaje de veracidad.
train_aux2 = train
train_aux2['words'] = clean_words
words_pairs = []

for sentence in train_aux2['words']:
    pairs = []
    for i in range(len(sentence)-1):
        pairs.append(sentence[i] + ' ' + sentence[i+1])
    words_pairs.append(pairs)
    
train_aux2['words'] = words_pairs
train_aux2 = train_aux2.explode('words')
train_aux2.head()

ValueError: Length of values does not match length of index

In [ ]:
train_aux2 = train_aux2.groupby('words').agg({'target':['sum','count']})
train_aux2 = train_aux2[train_aux2['target']['count']>min_repetitions]
train_aux2['veracity'] = train_aux2['target']['sum']/train_aux2['target']['count']

In [ ]:
#las palabras que son '' la cagan
train_aux2.head(10)

In [ ]:
train_aux2.reset_index(inplace=True)

In [ ]:
# Keywords más y menos veraces
train_aux3 = train
train_aux3 = train_aux3.groupby('keyword').agg({'target':['sum','count']})
train_aux3 = train_aux3[train_aux3['target']['count']>min_repetitions]
train_aux3['veracity'] = train_aux3['target']['sum']/train_aux3['target']['count']
train_aux3.head()

In [ ]:
train_aux3.reset_index(inplace=True)

In [ ]:
# Locaciones más y menos veraces
train_aux4 = train
train_aux4 = train_aux4.groupby('location').agg({'target':['sum','count']})
train_aux4 = train_aux4[train_aux4['target']['count']>min_repetitions]
train_aux4['veracity'] = train_aux4['target']['sum']/train_aux4['target']['count']
train_aux4.head()

In [ ]:
train_aux4.reset_index(inplace=True)

#### Parte final

In [ ]:
top_words = train_aux[train_aux['veracity']>0.9]['words']
worst_words = train_aux[train_aux['veracity']<0.1]['words']

In [ ]:
top_words_pairs = train_aux2[train_aux2['veracity']>0.9]['words']
worst_words_pairs = train_aux2[train_aux2['veracity']<0.1]['words']

In [ ]:
top_keywords = train_aux3[train_aux3['veracity']>0.9]['keyword']
worst_keywords = train_aux3[train_aux3['veracity']<0.1]['keyword']

In [ ]:
top_locations = train_aux4[train_aux4['veracity']>0.9]['location']
worst_locations = train_aux4[train_aux4['veracity']<0.1]['location']

In [ ]:
train = train.drop(columns='words')
train['long(char)'] = train['text'].str.len()

aux = []
for i in words:
    aux.append(len(i))

train['long(word)'] = aux
train.head()

In [ ]:
def count_text_ocurrences(texts, words):
    ocurrences = []
    for text in texts:
        count = 0
        for word in words:
            if word in text:
                count = count+1
        ocurrences.append(count)
    return ocurrences

def count_ocurrences(searched, series):
    ocurrences = []
    for element in searched:
        count = 0
        if element in series:
            count = count+1
        ocurrences.append(count)
    return ocurrences

def contains_char(texts, char1, char2=''):
    ocurrences = []
    for text in texts:
        count = 0
        if char1 in text:
            count = 1
        if (count==0)&(char2!=''):
            if char2 in text:
                count = 1
        ocurrences.append(count)
    return ocurrences

In [ ]:
train['top_words'] = count_text_ocurrences(train['text'], top_words)
train['worst_words'] = count_text_ocurrences(train['text'], worst_words)
train['top_words_pairs'] = count_text_ocurrences(train['text'], top_words_pairs)
train['worst_words_pairs'] = count_text_ocurrences(train['text'], worst_words_pairs)
train.head()

In [ ]:
train['top_keywords'] = count_ocurrences(train['keyword'], top_keywords)
train['worst_keywords'] = count_ocurrences(train['keyword'], worst_keywords)
train['top_locations'] = count_ocurrences(train['location'], top_locations)
train['worst_locations'] = count_ocurrences(train['location'], worst_locations)
train.head()

In [ ]:
train['arroba'] = contains_char(train['text'],'@')
train['singnosDeInterrogacion'] = contains_char(train['text'],'\?', '¿')
train['signosDeExclamacion'] = contains_char(train['text'],'!','¡')
train.head()

In [ ]:
train = train.drop(columns=['keyword','location','text'])
train = train.set_index('id')
train.head(20)

In [ ]:
col_rename = {'long(char)': 'longEnChars', 'long(word)': 'longEnWords', 'top_words': 'topWords', 'worst_words': 'worstWords', 'top_words_pairs': 'topWordsPairs',  
             'worst_words_pairs': 'worstWordsPairs', 'top_keywords': 'topKeywords', 'worst_keywords': 'worstKeywords', 'top_locations': 'topLocations', 
             'worst_locations': 'worstLocations'}
train = train.rename(columns= col_rename)
train.head(20)

### Entrenamiento del set de datos

In [ ]:
X, y = train.iloc[:,1:-1], train.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=465)

#### XGBoost

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 80)

In [ ]:
xg_reg.fit(X_train,y_train)

In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/xgboost.csv')
df.head()

In [ ]:
# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.2, 0.3, 0.4],
#         'max_depth': [10, 15, 20]
#         }
# grid_acc = GridSearchCV(xg_reg, param_grid = params)
# grid_acc.fit(X_train, y_train)
# y_pred_acc = grid_acc.predict(X_test)

# # New Model Evaluation metrics 
# print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred_acc.round())))
# print('Precision Score : ' + str(precision_score(y_test,y_pred_acc.round())))
# print('Recall Score : ' + str(recall_score(y_test,y_pred_acc.round())))
# print('F1 Score : ' + str(f1_score(y_test,y_pred_acc.round())))

#### RandomForest

In [ ]:
rf_model = RandomForestRegressor(random_state=13, n_estimators=80, max_depth=20)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/randomForest.csv')
df.head()

rf_model.feature_importances_
plt.figure(figsize=(17,9))
plt.bar(X_train.columns, rf_model.feature_importances_)
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.title('Importancia Features con RF')
plt.show()

#### LightGBM

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
     'objective': 'regression',
     'metric': 'rmse',
     'num_leaves': 5,
     'learning_rate': 0.1,
     'feature_fraction': 0.9,
 }

params = {
     'objective': 'regression',
 }

gbm = lgb.train(params,
                 lgb_train,
                 num_boost_round=100,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=10)

preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/lightgbm.csv')
df.head()

#### CatBoost

In [ ]:
model = CatBoostRegressor()
model.fit(X_train, y_train)
preds = model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/catboost.csv')
df.head()

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/logicRegression.csv')
df.head()

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 80)
rf_model = RandomForestRegressor(random_state=13, n_estimators=80, max_depth=20)

eclf2 = VotingRegressor(estimators=[
         ('xgb', xg_reg), ('rf', rf_model)])

In [ ]:
eclf2 = eclf2.fit(X, y)

In [ ]:
preds = eclf2.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))